In [ ]:
#%% Applying CatBoost

#Splitting training and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2, random_state = 0)


from catboost import CatBoostClassifier
categorical_var = np.where(x_train.dtypes != np.float)[0]
print('\nCategorical Variables indices : ',categorical_var)

In [ ]:
cat_model = CatBoostClassifier(iterations=200,learning_rate=0.5)
cat_model.fit(x_train,y_train,cat_features=categorical_var, plot=False)

cat_pred = cat_model.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
confusion_matrix = confusion_matrix(cat_pred,y_test)
print(confusion_matrix)
print("Accuracy:",accuracy_score(cat_pred,y_test))
print("Recall:",recall_score(cat_pred,y_test))
print("Precision:",precision_score(cat_pred,y_test))
print("F1 Score:",f1_score(cat_pred,y_test))

In [ ]:
#cat_model.get_feature_importance()
from catboost import Pool
def get_feature_imp_plot(method):
    
    if method == "Permutation":
        fi =  permutation_importances(cat_model, x_test, y_test, log_loss)
    
    elif method == "Baseline":
        fi = baseline_importance(cat_model, x_train, y_train, x_test, y_test, log_loss)
    
    elif method == "ShapeValues":
        shap_values = cat_model.get_feature_importance(Pool(x_test, label=y_test,cat_features=categorical_var), 
                                                                     type="ShapValues")
        shap_values = shap_values[:,:-1]
        shap.summary_plot(shap_values, x_test) 
        
    else:
        fi = cat_model.get_feature_importance(Pool(x_test, label=y_test,cat_features=categorical_var), 
                                                                     type=method)
        
    if method != "ShapeValues":
        feature_score = pd.DataFrame(list(zip(x_test.dtypes.index, fi )),
                                        columns=['Feature','Score'])

        feature_score = feature_score.sort_values(by='Score', ascending=False, inplace=False, kind='quicksort', na_position='last')

        plt.rcParams["figure.figsize"] = (12,7)
        ax = feature_score.plot('Feature', 'Score', kind='bar', color='c')
        ax.set_title("Feature Importance using {}".format(method), fontsize = 14)
        ax.set_xlabel("features")
        ax.set_ylabel("scores")
        plt.show()

get_feature_imp_plot(method="PredictionValuesChange")